In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

# Multi-agent Customer Support 

Here lets focus on these:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
#from utils import get_openai_api_key

from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

### Role Playing, Focus and Cooperation

- **Role Playing:** Both agents have been given a role, goal and backstory.
  
- **Focus:** Both agents have been prompted to get into the character of the roles they are playing.
  
- **Cooperation:** Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.


Note: here, we will set `allow_delegation = False`. If it's True, the agent can tranfer the task to other agent specialized in hanlding particular task.

In [8]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at HuggingFace (https://huggingface.co/) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

In [9]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

### Tools, Guardrails and Memory




In [7]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

**Possible Custom Tools**
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
... and more


we can also use these tools as shown:
```py
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

In [10]:
# this will scrape a page (only 1 URL for chat template on HF. 
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://huggingface.co/docs/transformers/main/en/chat_templating"
)

**Assigning Tools to AI Agents:**

* **Agent Level:**
    * Tools are available to the agent for all tasks.
    * Useful for general-purpose tools frequently used.

* **Task Level:**
    * Tools are restricted to specific tasks.
    * Better for specialized tools used infrequently or for specific scenarios.
    * Overrides agent-level tool assignments.


### Creating Tasks

Lets pass the Tool on the Task Level.


In [11]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

**çets build the Quality Assurance Reviewer agent**
- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [12]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Create Crew

Setting `memory=True` when putting the crew together enables Memory (short term, long term memories).

In [13]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

### Running the Crew

**Guardrails**
By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [14]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with using up a chat template "
               "and is there any automated pipelines for that? "
               "how can use external tools to get answer? "
               "Can you provide any template writing tips too?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with using up a chat template and is there any automated pipelines for that? how can use external tools to get answer? Can you provide any template writing tips too?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
I need to provide Andrew Ng from DeepLearningAI with the best support possible by fully addressing his questions about using chat templates, automated pipelines, and template writing tips. I should use the Read website content tool to gather information from the HuggingFace website.

Action: Read website content
Action Input: {"url": "https://huggingface.co/docs/transformers/main/en/chat_templating"} 

Templa

In [17]:
from IPython.display import Markdown
Markdown(result)

Dear Andrew Ng,

Thank you for reaching out with your questions regarding chat templates, automated pipelines, and template writing tips. I have gathered detailed information from the HuggingFace website to provide you with the best support possible.

1. Using Chat Templates:
Chat templates play a crucial role in converting conversations into a format that chat models can understand. To utilize chat templates effectively, you can create a list of messages with role and content keys and then apply them using the apply_chat_template() method. This method facilitates formatting conversations for model input and generating output that is ready for use.

2. Automated Pipelines for Chat:
Indeed, there are automated pipelines available for chat inputs, simplifying the process of using chat models. The TextGenerationPipeline class is designed to handle both chat templates and model generation. By initializing the pipeline and providing a list of messages, the pipeline takes care of tasks such as tokenization and template application, streamlining the workflow for you.

3. Template Writing Tips:
When crafting chat templates, it is essential to adhere to Jinja template syntax and implement best practices. To ensure optimal performance, remember to eliminate unnecessary whitespace, utilize for loops and if statements effectively, and make use of special variables like bos_token and eos_token for precise formatting. Additionally, consider compatibility with non-Python Jinja implementations by employing Jinja filters and standard syntax.

For more in-depth examples on leveraging chat templates in various scenarios such as training, tool use/function calling, and retrieval-augmented generation, as well as advanced tips on tool schemas and template editing, I recommend referring to the comprehensive content available on the HuggingFace website.

Should you have any further questions or require additional assistance, please do not hesitate to reach out. We are committed to helping you optimize your utilization of chat templates and enhance your experience with HuggingFace models.

Best regards,
[Your Name]
Senior Support Representative at HuggingFace